# Run Dense reconstruction

This notebook will guide you for running a dense reconstruction with LOFTR or RoMa starting from an existing reconstruction (e.g., carried out with SuperPoint+LightGlue that are more robust for image orientation).

This notebook will perform the dense matching with RoMa and it uses pycolmap to triangulate the points from the existing camera poses using pycolmap. To build a sparse reconstruction with SuperPoint+LightGlue, you can use the notebook `sfm_pipeline.ipynb`.


In [ ]:
import deep_image_matching as dim
import yaml
import pycolmap
from deep_image_matching.triangulation import db_from_existing_poses
from deep_image_matching.utils import OutputCapture

logger = dim.setup_logger("dim")

# Define the paraemters for the dense matching
params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "roma",
    "config_file": "../assets/example_cyprus/config_roma.yaml",
    "strategy": "matching_lowres",
    "quality": "medium",
    "tiling": "none",
    "skip_reconstruction": True,
    "force": True,
    "camera_options": "../assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}

# Build the configuration object
config = dim.Config(params)

Define the parameters for building the dense reconstruction starting from the known camera poses computed previously.


In [ ]:
# Define project directory
root_path = config.general["output_dir"].parent
image_dir = config.general["image_dir"]

# Path to the pre-computed COLMAP reconstuction with the knwon poses (set your own path)
sfm_path = root_path / "results_superpoint+lightglue_matching_lowres_quality_medium"
sfm_db_path = sfm_path / "database.db"
sfm_rec_path = sfm_path / "reconstruction"

# Path to the dense matching results to be triangulated
dense_path = config.general["output_dir"]
features_h5 = dense_path / "features.h5"
matches_h5 = dense_path / "matches.h5"
pair_file = dense_path / "pairs.txt"

# Path to the output for the dense matching
dense_db_path = dense_path / "database_dense.db"
model_path = dense_path / "dense_model"
model_path.mkdir(exist_ok=True, parents=True)

# Do geometric verification of the dense features (computing the epipolar error)
do_geometric_verification = True
max_error = 6

# Define trinagulation parameters
triang_min_angle = 0.1

In [ ]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

# Export in colmap format
database_path = config.general["output_dir"] / "database.db"
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_config_path=config.general["camera_options"],
)

In [ ]:
# Open sfm reconstruction with pycolmap
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create a new database with the dense features and the known camera poses
db_from_existing_poses(
    dense_db_path,
    features_h5,
    matches_h5,
    sfm_rec_path,
    pair_file,
    do_geometric_verification=do_geometric_verification,
    max_error=max_error,
)

In [ ]:
# Run the triangulation with the known camera poses

# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=triang_min_angle,
    ),
)
verbose = True

with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            dense_db_path,
            image_dir,
            model_path,
            options=opt,
        )

In [ ]:
# Export the reconstruction in ply (to be opened in CloudCompare or Meshlab)
reconstruction.export_PLY(model_path / "model.ply")

# Export the reconstruction in bundler format (to be imported in Metashape)
reconstruction.export_bundler(
    model_path / "bundler.out",
    model_path / "bundler_list.txt",
    skip_distortion=True,
)

Now you can open the dense reconstruction also with the COLMAP GUI.
